In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('ML').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [6]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [10]:
#create an instance of the model
lr = LinearRegression(featuresCol = 'features',
                      labelCol = 'label',predictionCol = 'predection')

In [11]:
#train the model
lrModel = lr.fit(training)

Exception ignored in: <function JavaWrapper.__del__ at 0x7fc7688ba310>
Traceback (most recent call last):
  File "/home/ahmad/Desktop/spark-3.2.0-bin-hadoop3.2/python/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LinearRegression' object has no attribute '_java_obj'


In [12]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [13]:
lrModel.intercept

0.14228558260358093

In [14]:
#spliting test and train data

In [15]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [19]:
#spliting the data into 70% training and 30% tetsing
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                344|
|   mean| 0.6483635077761702|
| stddev|   10.4103898792943|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                157|
|   mean|-0.6008646290147284|
| stddev|   10.0919149799843|
|    min|-23.487440120936512|
|    max|  23.52945433069272|
+-------+-------------------+



In [23]:
correct_model = lr.fit(train_data)

In [24]:
#evaluate on the test data, comparing our predections with the correct values in test data
test_results = correct_model.evaluate(test_data)

In [27]:
test_results.predictions.show()

+-------------------+--------------------+--------------------+
|              label|            features|          predection|
+-------------------+--------------------+--------------------+
|-23.487440120936512|(10,[0,1,2,3,4,5,...| -1.6690861999139948|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|  5.3641735626498175|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|  1.2089917302190551|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|  0.5514783972675898|
|-19.402336030214553|(10,[0,1,2,3,4,5,...| 0.33191210902303264|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|  2.9028978127072973|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|  3.0629565156594216|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|  1.2496115484432366|
|-15.951512565794573|(10,[0,1,2,3,4,5,...|  -2.398681231166365|
| -15.86200932757056|(10,[0,1,2,3,4,5,...|   1.743239855518799|
|-15.780685032623301|(10,[0,1,2,3,4,5,...|   4.103838036756712|
| -15.72351561304857|(10,[0,1,2,3,4,5,...|  1.4963414761455114|
|-15.437384793431217|(10,[0,1,2,3,4,5,..

In [28]:
unlabelled_data = test_data.select('features')

In [35]:
predictions = correct_model.transform(unlabelled_data)

In [37]:
predictions.collect()

[Row(features=SparseVector(10, {0: -0.5195, 1: 0.808, 2: 0.8499, 3: 0.0448, 4: -0.9032, 5: 0.284, 6: 0.964, 7: -0.0409, 8: 0.4419, 9: -0.736}), predection=-1.6690861999139948),
 Row(features=SparseVector(10, {0: 0.4798, 1: 0.02, 2: -0.8828, 3: 0.2755, 4: 0.0155, 5: 0.9653, 6: 0.6623, 7: -0.7708, 8: 0.1773, 9: 0.4782}), predection=5.3641735626498175),
 Row(features=SparseVector(10, {0: -0.4785, 1: 0.5204, 2: -0.2989, 3: -0.4626, 4: 0.5394, 5: 0.3932, 6: 0.1891, 7: 0.1312, 8: 0.4361, 9: 0.3954}), predection=1.2089917302190551),
 Row(features=SparseVector(10, {0: -0.9326, 1: -0.6411, 2: 0.9949, 3: 0.483, 4: -0.8736, 5: -0.3622, 6: 0.264, 7: 0.4553, 8: -0.9425, 9: 0.6251}), predection=0.5514783972675898),
 Row(features=SparseVector(10, {0: 0.4623, 1: -0.903, 2: 0.7443, 3: 0.3803, 4: 0.4069, 5: -0.5055, 6: -0.8686, 7: -0.0147, 8: -0.1363, 9: 0.8444}), predection=0.33191210902303264),
 Row(features=SparseVector(10, {0: -0.0177, 1: 0.5633, 2: 0.1423, 3: 0.0317, 4: -0.7815, 5: 0.3658, 6: 0.720